<a href="https://colab.research.google.com/github/ferkrum/web-scraper-tabela-wikipedia/blob/main/Web_Scraper_tabela_wikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Como obter conteúdo de uma tabela contida em uma página da Wikipedia.

Tabela usada disponível [aqui](https://pt.wikipedia.org/wiki/Lista_de_capitais_do_Brasil_por_%C3%A1rea).

Linguagem: Python

_libs: requests, BeautifulSoup_



## 1º: Importação de bibliotecas: 
1. "request" para fazer requisições de dados da web.
2. "BeautifulSoup" para fazer o processamento da informação (*parsing*) obtida pelo request.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## 2º: Request + parsing (beautifulSoup)

In [2]:
#obtém o conteúdo da página através do request 
response = requests.get('https://pt.wikipedia.org/wiki/Lista_de_capitais_do_Brasil_por_%C3%A1rea').content

In [3]:
#debug do que há em response
response

b'<!DOCTYPE html>\n<html class="client-nojs" lang="pt" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Lista de capitais do Brasil por \xc3\xa1rea \xe2\x80\x93 Wikip\xc3\xa9dia, a enciclop\xc3\xa9dia livre</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\\t.","\xc2\xa0\\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","janeiro","fevereiro","mar\xc3\xa7o","abril","maio","junho","julho","agosto","setembro","outubro","novembro","dezembro"],"wgRequestId":"b2ba8826-f38c-4644-bdb6-c6781a96c9cf","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Lista_de_capitais_do_Brasil_por_\xc3\xa1rea","wgTitle":"Lista de capitais do Brasil por \xc3\xa1rea","wgCurRevisionId":59645590,"wgRevisionId":59645590,"wgArticleId":2780611,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wg

In [4]:
# parsing com BeautifulSoup
soup = BeautifulSoup(response, 'html.parser')

In [5]:
soup

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="pt">
<head>
<meta charset="utf-8"/>
<title>Lista de capitais do Brasil por área – Wikipédia, a enciclopédia livre</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","janeiro","fevereiro","março","abril","maio","junho","julho","agosto","setembro","outubro","novembro","dezembro"],"wgRequestId":"b2ba8826-f38c-4644-bdb6-c6781a96c9cf","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Lista_de_capitais_do_Brasil_por_área","wgTitle":"Lista de capitais do Brasil por área","wgCurRevisionId":59645590,"wgRevisionId":59645590,"wgArticleId":2780611,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["!Artigos com ligações inativas","Listas de municípios 

A partir daqui devemos procurar pelos tags que queremos obter da página procurada.
Usamos a ferramenta "inspect" do navegador para identificar as tags.

**Modelo do dicionario de destino:**
dictCapitaisTabela[posicao] : { 'sede_do_governo' : sede_do_govermo, 'ibge' : ibge, 'uf' : uf, 'area' : area}

In [6]:
capitaisTabela = soup.find("tbody")

In [7]:
capitaisTabela

<tbody><tr style="background:#ececec;">
<th>Posição
</th>
<th>Sede de governo
</th>
<th>Código do IBGE
</th>
<th>Unidade federativa
</th>
<th>Área (km²)
</th></tr>
<tr>
<td align="center">1</td>
<td><a href="/wiki/Porto_Velho" title="Porto Velho">Porto Velho</a></td>
<td align="center">1100205</td>
<td><img alt="" class="thumbborder" data-file-height="1400" data-file-width="2000" decoding="async" height="14" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Bandeira_de_Rond%C3%B4nia.svg/20px-Bandeira_de_Rond%C3%B4nia.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Bandeira_de_Rond%C3%B4nia.svg/30px-Bandeira_de_Rond%C3%B4nia.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Bandeira_de_Rond%C3%B4nia.svg/40px-Bandeira_de_Rond%C3%B4nia.svg.png 2x" width="20"/> <a href="/wiki/Rond%C3%B4nia" title="Rondônia">Rondônia</a></td>
<td align="right">34 090,952
</td></tr>
<tr>
<td align="center">2</td>
<td><a href="/wiki/Manaus" title="Manaus">Manaus</a>

In [8]:
linhasTabela = capitaisTabela.find_all("tr")

In [9]:
len(linhasTabela)

28

## Inicializa dicionario de destino:

In [10]:
#inicializa dicionario final 
dictCapitaisTabela = {}

In [11]:
len(linhasTabela)

28

modelo do dicionario --> \\dictCapitaisTabela[posicao] : { "sede" : sede_do_govermo, 'ibge' : ibge, 'uf' : uf, 'area' : area}

In [12]:
#para cada linha da tabela, busca todas celulas
for i in linhasTabela:              #for para percorrer linhas da tabela
  celulasTabela = i.find_all("td")  #busca todos os tags "td" 
  try:                              #usamos try except porque a primeira linha da coluna não contém a tag "td", mas sim "th" e isso gerava um erro no programa
    dictCapitaisTabela[str.strip(celulasTabela[0].text)] = {"sede": str.strip(celulasTabela[1].text), "ibge": str.strip(celulasTabela[2].text), "uf": str.strip(celulasTabela[3].text), "area": str.strip(celulasTabela[4].text).replace(u'\xa0', u' ')} #foi necessário usar replace no campo de "área" porque havia um hard space na string
  except:
    print("") 

In [13]:
#debug testando o dicionario
dictCapitaisTabela["1"]["area"]

'34 090,952'

## Criando dataframe e exportando arquivo. csv

In [14]:
dfCapitais = pd.DataFrame.from_dict(dictCapitaisTabela).T

In [15]:
dfCapitais.to_csv("capitais.csv")

In [16]:
dfCapitais

,sede,ibge,uf,area
1,Porto Velho,1100205,Rondônia,"34 090,952"
2,Manaus,1302603,Amazonas,"11 401,092"
3,Rio Branco,1200401,Acre,"8 834,942"
4,Campo Grande,5002704,Mato Grosso do Sul,"8 082,978"
5,Macapá,1600303,Amapá,"6 563,849"
6,Brasília,5300108,Distrito Federal,"5 760,783"
7,Boa Vista,1400100,Roraima,"5 687,037"
8,Cuiabá,5103403,Mato Grosso,"3 266,538"
9,Palmas,1721000,Tocantins,"2 227,444"
10,São Paulo,3550308,São Paulo,"1 521,110"
